## Dashboard for exploring glider trajectory relative to surface properties

Here we create a panel dashboard that allows us to explore the lat-lon-time trajectory of the glider relative to a few surface fields.

In [ ]:
import hvplot.xarray 
import numpy as np
import panel as pn
import xarray as xr
import holoviews as hv
import geoviews as gv
from holoviews import opts
from holoviews import streams
import matplotlib.pyplot as plt
import geopandas
import pandas as pd
import hvplot.pandas  # noqa
import param
import cartopy.crs as ccrs

### Load in surface variables

In [ ]:
data_dir = './data/'
ds_ssh  = xr.open_dataset(data_dir + 'SSH_sogos_processed.nc')
ds_fsle = xr.open_dataset(data_dir + 'FSLE_sogos_processed.nc')

ds_659_locs = xr.open_dataset(data_dir + '659_locs.nc')
ds_660_locs = xr.open_dataset(data_dir + '660_locs.nc')

In [ ]:
ds_ssh  = ds_ssh.assign_coords(days = ds_ssh.days)
ds_fsle = ds_fsle.assign_coords(days = ds_fsle.days)

In [ ]:
# convert to pandas dataframe as it is much easier to handle in holoviz for traj data.
ds_659_pd = ds_659_locs.to_dataframe()
ds_660_pd = ds_660_locs.to_dataframe()
#ds_659_gpd = geopandas.GeoDataFrame(ds_659_locs.to_dataframe())
#ds_660_gpd = geopandas.GeoDataFrame(ds_660_locs.to_dataframe())

### Main part

In [ ]:
#glider_nums = ['sg659', 'sg660']
glider_map = {
            'sg659': {'loc': ds_659_pd},
            'sg660': {'loc': ds_660_pd},
        }

In [ ]:
surface_var_map = {
    'SSH' : ds_ssh['adt'],
    'SSHA': ds_ssh['sla'],
    'FSLE': ds_fsle['fsle_max']
    }

In [ ]:
class TrajectoryPlot(param.Parameterized):
    surface_var = param.Selector(surface_var_map.keys(), default='SSH',
                                label='Surface Field', precedence=0)
    glider_num = param.Selector(glider_map.keys(), default='sg659',
                                label='Glider Num', precedence=0)
    time_slider = param.Range(label='Days in 2019', 
                             bounds=(119, 205), 
                             default=(119, 135), precedence=3)
    alpha_slider = param.Magnitude(label='Transparency', precedence=4)
    
    # Function to plot trajectories
    @param.depends('glider_num', 'time_slider')
    def plot_traj(self):
        time_rng = self.time_slider
        
        traj = {}
        for glid in glider_nums:
            ds = glider_map[glid]['loc']
            ds_tsel = ds.loc[(ds.days>time_rng[0]) & (ds.days<time_rng[1])]
        
            traj[glid] = ds_tsel.hvplot.points(geo=True,  x='longitude', y='latitude', 
                                               hover=True, hover_cols=['days'], 
                                               size=1)
            # setting PlateCarree as projection makes the hover cols show up properly, but then the bathy disappears.

        traj[self.glider_num].opts(size=2.5)
        
        return traj['sg659']*traj['sg660']
    
    # Function to plot tiles
    def surf_tiles(self):
        gebco_tiles = 'https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/GEBCO_basemap_NCEI/MapServer/tile/{Z}/{Y}/{X}'
        return gv.WMTS( gebco_tiles )
    
    # function to plot vector field
    @param.depends('time_slider')
    def surf_vec(self):
        time_sel = self.time_slider[1] # show map for last day on time slider
        
        return ds_ssh.where(ds_ssh.days==time_sel, drop=True).squeeze('time'
                    ).hvplot.vectorfield(x='longitude', y='latitude', angle='angle', mag='mag',
                                        geo=True, hover=False).opts(magnitude='mag')
    
    #function to plot surface field 
    @param.depends('surface_var', 'time_slider', 'alpha_slider')
    def plot_surface(self):
        time_sel = self.time_slider[1] # show map for last day on time slider
        
        ds_all = surface_var_map[self.surface_var]
        ds = ds_all.where(ds_all.days==time_sel, drop=True).squeeze('time')
        if self.surface_var == 'FSLE':    
            surf_plot = ds.hvplot.image(geo=True)
            surf_plot.opts(clim=(-0.6,0), cmap='Blues_r', clabel='FSLE')
        elif self.surface_var == 'SSH':
            surf_plot = ds.hvplot.image(geo=True)
            surf_plot.opts(clim=(-1,0), cmap='cividis', clabel='SSH')
        else: 
            surf_plot = ds.hvplot.image(geo=True)
            surf_plot.opts(clim=(-0.3,0.3), cmap='RdBu_r', clabel='SSHA')
        
        surf_plot.opts(alpha=self.alpha_slider)

        return surf_plot
        
    def view(self):
        return hv.DynamicMap(self.plot_surface)*hv.DynamicMap(self.surf_tiles
                                        )*hv.DynamicMap(self.surf_vec)*hv.DynamicMap(self.plot_traj)

In [ ]:
test = TrajectoryPlot()

In [ ]:
pn.Row(test.param, test.view())

### 2 Class approach with separate parameters and plotting

In [ ]:
class TrajectoryPlot_params(param.Parameterized):
    surface_var = param.Selector(surface_var_map.keys(), default='SSH',
                                label='Surface Field', precedence=0)
    glider_num = param.Selector(glider_map.keys(), default='sg659',
                                label='Glider Num', precedence=0)
    time_slider = param.Range(label='Days in 2019', 
                             bounds=(119, 205), 
                             default=(119, 135), precedence=3)
    alpha_slider = param.Magnitude(label='Transparency', precedence=4)

class TrajectoryPlot2(TrajectoryPlot_params):
    # Function to plot trajectories
    @param.depends('glider_num', 'time_slider')
    def plot_traj(self):
        time_rng = self.time_slider
        
        traj = {}
        for glid in glider_nums:
            ds = glider_map[glid]['loc']
            ds_tsel = ds.loc[(ds.days>time_rng[0]) & (ds.days<time_rng[1])]
        
            traj[glid] = ds_tsel.hvplot.points(geo=True,  x='longitude', y='latitude', 
                                               hover=True, hover_cols=['days'], 
                                               size=1)
            # setting PlateCarree as projection makes the hover cols show up properly, but then the bathy disappears.

        traj[self.glider_num].opts(size=2.5)
        
        return traj['sg659']*traj['sg660']
    
    # Function to plot tiles
    def surf_tiles(self):
        gebco_tiles = 'https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/GEBCO_basemap_NCEI/MapServer/tile/{Z}/{Y}/{X}'
        return gv.WMTS( gebco_tiles )
    
    # function to plot vector field
    @param.depends('time_slider')
    def surf_vec(self):
        time_sel = self.time_slider[1] # show map for last day on time slider
        
        return ds_ssh.where(ds_ssh.days==time_sel, drop=True).squeeze('time'
                    ).hvplot.vectorfield(x='longitude', y='latitude', angle='angle', mag='mag',
                                        geo=True, hover=False).opts(magnitude='mag')
    
    #function to plot surface field 
    @param.depends('surface_var', 'time_slider', 'alpha_slider')
    def plot_surface(self):
        time_sel = self.time_slider[1] # show map for last day on time slider
        
        ds_all = surface_var_map[self.surface_var]
        ds = ds_all.where(ds_all.days==time_sel, drop=True).squeeze('time')
        if self.surface_var == 'FSLE':    
            surf_plot = ds.hvplot.image(geo=True)
            surf_plot.opts(clim=(-0.6,0), cmap='Blues_r', clabel='FSLE')
        elif self.surface_var == 'SSH':
            surf_plot = ds.hvplot.image(geo=True)
            surf_plot.opts(clim=(-1,0), cmap='cividis', clabel='SSH')
        else: 
            surf_plot = ds.hvplot.image(geo=True)
            surf_plot.opts(clim=(-0.3,0.3), cmap='RdBu_r', clabel='SSHA')
        
        surf_plot.opts(alpha=self.alpha_slider)

        return surf_plot
        
    def view(self):
        return hv.DynamicMap(self.plot_surface)*hv.DynamicMap(self.surf_tiles
                                        )*hv.DynamicMap(self.surf_vec)*hv.DynamicMap(self.plot_traj)

In [ ]:
test2 = TrajectoryPlot2()

In [ ]:
pn.Row(test2.param, test2.view())